In [ ]:
import cv2
import pandas as pd
import json
import matplotlib.pyplot as plt

# Function to add a translucent box on the image
def add_translucent_box(image, bbox):
    # Extract the coordinates and dimensions of the bbox
    x, y, width, height = bbox
    x, y, width, height = int(x), int(y), int(width), int(height)

    # Create a rectangle overlay with transparency
    overlay = image.copy()
    cv2.rectangle(overlay, (x, y), (x+width, y+height), (255, 0, 0), -1)  # Red rectangle with full opacity

    # Apply the overlay. 0.3 is the transparency factor
    image_new = cv2.addWeighted(overlay, 0.3, image, 0.7, 0)

    return image_new

# Load your CSV file into a DataFrame
df = pd.read_csv('FracAtlas/dataset.csv')

# Filter the DataFrame
filtered_df = df[(df['leg'] == 1) & (df['hip'] == 0) & (df['multiscan'] == 0) & (df['frontal'] == 1) & (df['hardware'] == 0)]

# Load and parse the COCO JSON file
with open(r'FracAtlas/Annotations/COCO JSON/COCO_fracture_masks.json', 'r') as file:  # Update the path to your JSON file
    data = json.load(file)

# Directory where the images are stored (update this path as necessary)
image_directory = 'FracAtlas/images/Fractured/'

# Process each image based on the annotations
count = 0
fracs = 0
imgs = []
for annotation in data['annotations']:
    image_id = annotation['image_id']
    bbox = annotation['bbox']
    segmentation = annotation['segmentation']
    
    # Find the corresponding image file
    image_info = next((item for item in data['images'] if item['id'] == image_id), None)
    if image_info:
        # Get all rows that match the file_name
        matches_df = filtered_df[filtered_df['image_id'] == image_info['file_name']]

        # Check if there are any matches
        if not matches_df.empty:
            corresponding_row = matches_df.iloc[0]  # Safely get the first row
            
            file_name = image_directory + image_info['file_name']
                    
            # Load the image
            img = cv2.imread(file_name, cv2.IMREAD_COLOR)
            if img is not None:
                # for seg in segmentation:
                #     pts = np.array(seg, np.int32).reshape((-1, 1, 2))
                #     cv2.polylines(img, [pts], isClosed=True, color=(0, 0, 255), thickness=2)  # Red color

                # Show the image using Matplotlib
                plt.figure(figsize=(10, 8))
                plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                plt.axis('off')
                plt.show()

                if corresponding_row['fractured']: fracs += 1
                count += 1
            else:
                print(f"Image {file_name} not found.")
            if count > 1:
                break